En este databrick vamos a ver en detalle los nuevos funcionalidades disponibles que tiene Delta Lake y sus ventajas:

In [0]:
# Create data DataFrame
data = spark.range(0, 5)
# Write the data DataFrame to /delta location
data.write.format("delta").mode('overwrite').save("/dbfs/delta")
#De esta forma guardamos en el metastore la tabla delta con los delta logs para el versionado de la tabla.

In [0]:
#También es posible crear una tabla Hive con saveAsTable() en Spark
data.write.format("delta").mode('overwrite').saveAsTable("tabla2")


In [0]:
#En caso de querer añadir nuevas filas a una tabla delta, es necesario usar el modo append
data2 = spark.range(6, 10)
data2.write.format("delta").mode("append").save("/dbfs/delta")
#El modo append en Spark es extremadamente costoso a nivel de cómputo. Con Delta Table se realiza en segundos

In [0]:
# Leer un delta table
dfDelta = spark.read.format("delta").load("/dbfs/delta")
dfDelta.count()

Out[14]: 9

Otra forma de crear una tabla delta pero en SQL

In [0]:
%sql 
CREATE TABLE student (id INT, name STRING, age INT)
    USING DELTA
    PARTITIONED BY (age)
    LOCATION '/event';

### Explorar Delta Logs

Se puede observar 3 tipos de ficheros:

- Optimization: Para optimizar todo el proceso de carga de datos en la tabla Delta
- CRC: Aplica una técnica para verificar la integridad de los datos y deteción de errores con muy poc cómputo
- JSON: Aparece en Objetos Json todo el historial de las versiones aplicadas de la tabla Delta. Tiene un tiempo de retención fijada, por lo que se borra información que tenga un tiempo de vida mayor al definido

In [0]:
%fs
ls /user/hive/warehouse/tabla2/_delta_log

path,name,size
dbfs:/user/hive/warehouse/tabla2/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/user/hive/warehouse/tabla2/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/user/hive/warehouse/tabla2/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/user/hive/warehouse/tabla2/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/user/hive/warehouse/tabla2/_delta_log/00000000000000000000.json,00000000000000000000.json,2431


In [0]:
#Visualizar el contenido del historial de una tabla en formato json
j0 = spark.read.json("dbfs:/user/hive/warehouse/tabla2/_delta_log/00000000000000000000.json")

## Historial de una tabla Delta

En este apartado podemos ver en práctica comandos para describir una tabla Delta:

Datos curiosos: 
- Por defecto Databricks únicamente almacena el historial de 30 días por defecto los datos de la tabla delta, para que no crezca indefinídamente el tamaño de los datos.
- Con config spark.databricks.delta.deletedFileRetentionDuration puedes controlar el tiempo que pase para eliminar de forma permanente los ficheros snappy eliminados de la tabla delta. Por defecto son 7 días. WARNING: Databricks no recomienda definir un valor menor a 7 días por temas de snapshots y datos concurrentes de otros usuarios que puedan ocasionar una corrupción de los datos.

In [0]:
%sql
-- Review history by Delta table file path
DESCRIBE HISTORY 'dbfs:/user/hive/warehouse/tabla2';


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-07-28T11:12:53.000+0000,7524590642868617,jin.wang@bosonit.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4339904306915146),0728-110121-pinky983,null,WriteSerializable,true,"Map(numFiles -> 6, numOutputBytes -> 2911, numOutputRows -> 5)",null


In [0]:
%sql
describe detail tabla2

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,139a61c5-c30c-496b-a998-b121e840659d,default.tabla2,null,dbfs:/user/hive/warehouse/tabla2,2021-07-28T11:12:48.399+0000,2021-07-28T11:12:53.000+0000,List(),6,2911,Map(),1,2


In [0]:
%python
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/tabla2')
fullHistoryDF = deltaTable.history()
fullHistoryDF.show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
version| timestamp| userId| userName| operation| operationParameters| job| notebook| clusterId|readVersion| isolationLevel|isBlindAppend| operationMetrics|userMetadata|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
 0|2021-07-28 11:12:53|7524590642868617|jin.wang@bosonit.com|CREATE TABLE AS S...|{isManaged -> tru...|null|{4339904306915146}|0728-110121-pinky983| null|WriteSerializable| true|{numFiles -> 6, n...| null|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+

## Vacuum 
Datos curiosos:
- Para usar VACUUM de forma paralela, es necesario configurar el spark session con spark.databricks.delta.vacuum.parallelDelete.enabled" a "true"

In [0]:
%sql
VACUUM 'dbfs:/user/hive/warehouse/tabla2'

path
dbfs:/user/hive/warehouse/tabla2


In [0]:
%python
from delta.tables import *
# vacuum files in path-based tables
deltaTable = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/tabla2')
# vacuum files in metastore-based tables
deltaTable = DeltaTable.forName(spark, 'tabla2')
# vacuum files in path-based table by default retention threshold
deltaTable.vacuum()

Out[8]: DataFrame[]

## Generar un fichero de manifiesto

Es posible habilitar la ejecución de consultas a una tabla delta sin tener un sistema compatible con Spark. Para ello, ejecutarán un manifest file:

In [0]:
%python
# Generate manifest
deltaTable = DeltaTable.forPath(spark, '/user/hive/warehouse/tabla2')
deltaTable.generate("symlink_format_manifest")


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3613977498279290> in <module> 
 1 # Generate manifest 
 2 deltaTable = DeltaTable . forPath ( spark , '/user/hive/warehouse/tabla2' ) 
 ----> 3 deltaTable . generate ( "symlink_format_manifest" ) 

 /databricks/spark/python/delta/tables.py in generate (self, mode) 
 76 . . note : : Evolving
 77 """
 ---> 78 self . _jdt . generate ( mode ) 
 79 
 80 @ since ( 0.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o621.generate.
: java.lang.UnsupportedOperationException: com.databricks.backend.daemon.data.client.DBFSV1.resolvePathOnPhysicalStorage(path: Path)
	at com.databricks.tahoe.store.EnhancedDatabricksFileSystemV1.resolvePathOnPhysicalStorage(EnhancedFileSystem.scala:203)
	at com.databricks.tahoe.store.S3LogStoreBase.resolvePathOnPhysicalStorage(S3LogStore.scala:272)
	at com.databricks.tahoe.store.DelegatingLogStore.resolvePathOnPhysicalStorage(DelegatingLogStore.scala:132)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifestImplEdge.isS3$lzycompute$1(GenerateSymlinkManifestImplEdge.scala:66)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifestImplEdge.isS3$1(GenerateSymlinkManifestImplEdge.scala:64)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifestImplEdge.assertFileSystemSupported(GenerateSymlinkManifestImplEdge.scala:73)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifestImplEdge.generateFullManifest(GenerateSymlinkManifestImplEdge.scala:38)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifestImplEdge.generateFullManifest$(GenerateSymlinkManifestImplEdge.scala:37)
	at com.databricks.sql.transaction.tahoe.hooks.GenerateSymlinkManifest$.generateFullManifest(GenerateSymlinkManifest.scala:43)
	at com.databricks.sql.transaction.tahoe.commands.DeltaGenerateCommand$.$anonfun$modeNameToGenerationFunc$1(DeltaGenerateCommand.scala:69)
	at com.databricks.sql.transaction.tahoe.commands.DeltaGenerateCommand$.$anonfun$modeNameToGenerationFunc$1$adapted(DeltaGenerateCommand.scala:69)
	at com.databricks.sql.transaction.tahoe.commands.DeltaGenerateCommand.$anonfun$run$1(DeltaGenerateCommand.scala:55)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1258)
	at com.databricks.sql.transaction.tahoe.commands.DeltaGenerateCommand.run(DeltaGenerateCommand.scala:38)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:233)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3802)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:126)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQL

## Convert a Parquet table to a Delta table
Convertir una tabla parquet a delta es bastante trivial, ya que de por sí, la tabla delta ya está en formato parquet. Únicamente es necesario crear un registro de los versiones de la tabla para tener un seguimiento de estos ficheros parquet.

In [0]:
# Create data DataFrame
data2 = spark.range(0, 5)
# Write the data DataFrame to /delta location
data2.write.parquet("tablaParquet2.parquet")
#De esta forma guardamos en el metastore la tabla delta con los delta logs para el versionado de la tabla.

In [0]:
%sql
CREATE TABLE parquetToDelta USING PARQUET OPTIONS ( path 'dbfs:/tablaParquet2.parquet');

CONVERT TO DELTA parquetToDelta;

In [0]:
%python
from delta.tables import *
# Convert non partitioned parquet table at path '<path-to-table>'
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`dbfs:/tablaParquet2.parquet`")

## Restaurar una versión anterior en una tabla Delta

Aparte de poder observar el historial de la tabla, también podemos volver a versiones anteriores

In [0]:
%sql
-- Restore table as of version 0
RESTORE tabla2 VERSION AS OF 0


table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
2911,6,6,6,2911,2911


## Tablas clonadas: Shadow Clone y Deep Clone

In [0]:
%sql
-- Shallow clone table
CREATE OR REPLACE TABLE default.shadowClone SHALLOW CLONE default.tabla2 LOCATION '/temporalClones/shadowClone'

source_table_size,source_num_of_files,num_removed_files,num_copied_files,removed_files_size,copied_files_size
2911,6,0,0,0,0


In [0]:
%sql
select *, input_file_name() from shadowClone

id,input_file_name()
0,dbfs:/user/hive/warehouse/tabla2/part-00001-a2ac427c-e60d-4c0e-a6c7-946e849e128a-c000.snappy.parquet
1,dbfs:/user/hive/warehouse/tabla2/part-00003-fe449a1b-c922-44ee-8e41-40a0a37ee077-c000.snappy.parquet
2,dbfs:/user/hive/warehouse/tabla2/part-00004-7509cb85-a23f-46d4-93c8-a23d314dafcf-c000.snappy.parquet
3,dbfs:/user/hive/warehouse/tabla2/part-00006-4c8f42bf-8f5c-4a00-84d6-d11caeb6845f-c000.snappy.parquet
4,dbfs:/user/hive/warehouse/tabla2/part-00007-a40b1fa4-9ebc-4225-90ba-5cb3bc196d61-c000.snappy.parquet


In [0]:
dfShadow = spark.read.format("delta").load("dbfs:/user/hive/warehouse/shadowclone")
dfShadow.write.format("delta").mode('overwrite').saveAsTable("shadowClone")

In [0]:
%sql
-- Deep clone table
CREATE OR REPLACE TABLE default.deepClone DEEP CLONE default.tabla2 LOCATION '/temporalClones/deepClone'

source_table_size,source_num_of_files,num_removed_files,num_copied_files,removed_files_size,copied_files_size
2911,6,0,6,0,2911


In [0]:
%sql
select *, input_file_name() from deepClone

id,input_file_name()
0,dbfs:/temporalClones/deepClone/part-00001-a2ac427c-e60d-4c0e-a6c7-946e849e128a-c000.snappy.parquet
1,dbfs:/temporalClones/deepClone/part-00003-fe449a1b-c922-44ee-8e41-40a0a37ee077-c000.snappy.parquet
2,dbfs:/temporalClones/deepClone/part-00004-7509cb85-a23f-46d4-93c8-a23d314dafcf-c000.snappy.parquet
3,dbfs:/temporalClones/deepClone/part-00006-4c8f42bf-8f5c-4a00-84d6-d11caeb6845f-c000.snappy.parquet
4,dbfs:/temporalClones/deepClone/part-00007-a40b1fa4-9ebc-4225-90ba-5cb3bc196d61-c000.snappy.parquet
